### This notebook demonstrates how to use PhyloGPN to obtain log likelihoods, substitution rates, and embeddings.

In [1]:
from typing import List
import yaml
import torch

import sys
import os
sys.path.append(os.path.abspath("../src"))
from phylogpn import RCEByteNet

with open(f"../PhyloGPN/config.yaml", "r") as f:
    config = yaml.safe_load(f)

dilation_rates = [config["kernel_size"]**i for i in range(config["stack_size"])] * config["num_stacks"]

# The `involution_indices` args are to specify the indices of complementary bases
# Implicitly they also specify the vocab size and the number of outputs
model_args = {
    "input_involution_indices": [3, 2, 1, 0, 4, 5],
    "output_involution_indices": [3, 2, 1, 0],
    "dilation_rates": dilation_rates,
    "outer_dim": config["outer_dim"],
    "inner_dim": config["inner_dim"],
    "kernel_size": config["kernel_size"],
    "pad_token_idx": 5,
}
model = RCEByteNet(**model_args)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load("../PhyloGPN/checkpoint.pt", map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])

model

RCEByteNet(
  (embedding): ParametrizedEmbedding(
    6, 960, padding_idx=5
    (parametrizations): ModuleDict(
      (weight): ParametrizationList(
        (0): IEWeight()
      )
    )
  )
  (blocks): Sequential(
    (0): RCEByteNetBlock(
      (layers): Sequential(
        (0): GroupNorm(1, 960, eps=1e-05, affine=True)
        (1): GELU(approximate='none')
        (2): ParametrizedConv1d(
          960, 480, kernel_size=(1,), stride=(1,)
          (parametrizations): ModuleDict(
            (weight): ParametrizationList(
              (0): RCEWeight()
            )
            (bias): ParametrizationList(
              (0): IEBias()
            )
          )
        )
        (3): GroupNorm(1, 480, eps=1e-05, affine=True)
        (4): GELU(approximate='none')
        (5): ParametrizedConv1d(
          480, 480, kernel_size=(5,), stride=(1,)
          (parametrizations): ModuleDict(
            (weight): ParametrizationList(
              (0): RCEWeight()
            )
            (b

In [24]:
vocab = {x: i for i, x in enumerate("ACGTN")}

def tokenize(seq_list: List[str]):
    tensors = []

    for seq in seq_list:
        tensors.append(torch.tensor([vocab.get(x, 5) for x in seq], dtype=torch.long))

    return torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True, padding_value=5)

seq = "ACGTN" * 100
assert len(seq) >= 481
input_tensor = tokenize([seq]).to(device)
output_tensor = model(input_tensor)
assert output_tensor.shape[1] == len(seq) - 481 + 1

In [25]:
from IPython.display import display
import pandas as pd
import torch.nn.functional as F

likelihood_tensor = F.softmax(output_tensor, dim=2).squeeze(0)
likelihood_df = pd.DataFrame(likelihood_tensor.numpy(force=True), columns=list("ACGT"))
display(likelihood_df)

,A,C,G,T
0,0.680490,0.058482,0.202242,0.058787
1,0.045694,0.388994,0.044250,0.521062
2,0.533812,0.044782,0.376085,0.045321
3,0.063520,0.219829,0.062750,0.653901
4,0.220344,0.288824,0.281353,0.209479
5,0.680490,0.058482,0.202242,0.058787
6,0.045694,0.388994,0.044250,0.521062
7,0.533812,0.044782,0.376085,0.045321
8,0.063520,0.219829,0.062750,0.653901
9,0.220344,0.288824,0.281353,0.209479


In [26]:
rate_tensor = output_tensor.exp().squeeze(0)
rate_df = pd.DataFrame(rate_tensor.numpy(force=True), columns=list("ACGT"))
display(rate_df)

,A,C,G,T
0,38.388813,3.299152,11.409155,3.316384
1,3.854062,32.809608,3.732229,43.948788
2,44.534679,3.736043,31.375879,3.780991
3,3.492544,12.086968,3.450221,35.953709
4,5.657725,7.416061,7.224227,5.378751
5,38.388813,3.299152,11.409155,3.316384
6,3.854062,32.809608,3.732229,43.948788
7,44.534679,3.736043,31.375879,3.780991
8,3.492544,12.086968,3.450221,35.953709
9,5.657725,7.416061,7.224227,5.378751


In [27]:
embedding_tensor = model.encode(input_tensor)
embedding_tensor.shape

torch.Size([1, 20, 960])